In [2]:
import pandas as pd

In [8]:
df = pd.read_excel('krx_stocklist.xlsx')

/opt/anaconda3/envs/finance-agent/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [12]:
df = df[['한글 종목약명', '단축코드', '시장구분']]

In [13]:
df

,한글 종목약명,단축코드,시장구분
0,마이크로컨텍솔,098120,KOSDAQ
1,포스코엠텍,009520,KOSDAQ GLOBAL
2,AJ네트웍스,095570,KOSPI
3,AK홀딩스,006840,KOSPI
4,BGF리테일,282330,KOSPI
...,...,...,...
2870,흥국화재우,000545,KOSPI
2871,흥국화재,000540,KOSPI
2872,흥아해운,003280,KOSPI
2873,희림,037440,KOSDAQ


In [14]:
df['시장구분'].value_counts()

시장구분
KOSDAQ           1744
KOSPI             962
KONEX             117
KOSDAQ GLOBAL      52
Name: count, dtype: int64

In [16]:
df['ticker'] = ""

/var/folders/tx/yl47_gj55sg2qxzspxx7jy8m0000gn/T/ipykernel_62638/2982549004.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ticker'] = ""


In [17]:
for index, row in df.iterrows():
    if 'KOSDAQ' in row['시장구분']:
        df.at[index, 'ticker'] = row['단축코드'] + '.KQ'
    elif 'KOSPI' in row['시장구분']:
        df.at[index, 'ticker'] = row['단축코드'] + '.KS'
    else:
        df.drop(index, inplace=True)

In [19]:
df.columns = ['company_name', 'stock_code', 'market', 'ticker']

In [21]:
df.to_csv('krx_tickers.csv', index=False)

In [1]:
from tqdm import tqdm
import yfinance as yf
import pandas as pd
import time

import pandas as pd
import numpy as np
from sqlalchemy import create_engine, text

In [ ]:
ticker_df = pd.read_csv('krx_tickers.csv')
tickers = ticker_df['ticker'].tolist() 
tickers = list(set(tickers)) # 중복제거

combined = []

df = yf.download(tickers, period="5y", interval="1d", auto_adjust=False)

df_tidy = df.stack(level=1, future_stack=True).reset_index()
df_tidy = df_tidy.rename(columns={"Ticker": "ticker",
                                  "Date": "date",
                                  "Adj Close": "adj_close",
                                  "Close": "close",
                                  "High": "high",
                                  "Low": "low",
                                  "Open": "open",
                                  "Volume": "volume"
                                })
df_tidy.columns = ['date', 'ticker', 'adj_close', 'close', 'high', 'low', 'open', 'volume']

[*********************100%***********************]  2758 of 2758 completed

6 Failed downloads:
['099520.KQ']: YFPricesMissingError('possibly delisted; no price data found  (period=5y) (Yahoo error = "No data found, symbol may be delisted")')
['336370.KS', '019680.KS']: Timeout('Failed to perform, curl: (28) Connection timed out after 10002 milliseconds. See https://curl.se/libcurl/c/libcurl-errors.html first for more details.')
['079370.KQ']: Timeout('Failed to perform, curl: (28) Operation timed out after 10002 milliseconds with 21579 bytes received. See https://curl.se/libcurl/c/libcurl-errors.html first for more details.')
['005950.KS']: Timeout('Failed to perform, curl: (28) Connection timed out after 10000 milliseconds. See https://curl.se/libcurl/c/libcurl-errors.html first for more details.')
['102370.KQ']: Timeout('Failed to perform, curl: (28) Connection timed out after 10001 milliseconds. See https://curl.se/libcurl/c/libcurl-errors.html first for more details.')


In [4]:
df_tidy

,date,ticker,adj_close,close,high,low,open,volume
0,2020-08-11,000020.KS,22344.568359,24400.0,34450.0,23750.0,30350.0,11121453.0
1,2020-08-11,000040.KS,702.000000,702.0,734.0,697.0,711.0,1184051.0
2,2020-08-11,000050.KS,10962.075195,11750.0,11950.0,11700.0,11950.0,18600.0
3,2020-08-11,000070.KS,58903.398438,73000.0,73200.0,71600.0,72300.0,25983.0
4,2020-08-11,000075.KS,40572.058594,54300.0,54700.0,53500.0,54300.0,3391.0
...,...,...,...,...,...,...,...,...
3378545,2025-08-11,950170.KQ,6340.000000,6340.0,6450.0,6210.0,6250.0,112488.0
3378546,2025-08-11,950190.KQ,10040.000000,10040.0,10290.0,9720.0,10290.0,21528.0
3378547,2025-08-11,950200.KQ,3705.000000,3705.0,3770.0,3605.0,3770.0,25996.0
3378548,2025-08-11,950210.KS,14690.000000,14690.0,15380.0,14640.0,15050.0,253651.0


In [5]:
df_tidy.isnull().sum()

date              0
ticker            0
adj_close    397397
close        397397
high         397397
low          397397
open         397397
volume       397397
dtype: int64

In [10]:
df_tidy[df_tidy["adj_close"].isnull()]

,date,ticker,adj_close,close,high,low,open,volume
35,2020-08-11,0004Y0.KQ,NaN,NaN,NaN,NaN,NaN,NaN
65,2020-08-11,00104K.KS,NaN,NaN,NaN,NaN,NaN,NaN
180,2020-08-11,0030R0.KS,NaN,NaN,NaN,NaN,NaN,NaN
219,2020-08-11,0037T0.KQ,NaN,NaN,NaN,NaN,NaN,NaN
236,2020-08-11,0041J0.KQ,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
3378369,2025-08-11,448830.KQ,NaN,NaN,NaN,NaN,NaN,NaN
3378416,2025-08-11,457630.KQ,NaN,NaN,NaN,NaN,NaN,NaN
3378443,2025-08-11,464440.KQ,NaN,NaN,NaN,NaN,NaN,NaN
3378497,2025-08-11,478780.KQ,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
dat = yf.Ticker("336370.KS")
dat.history()

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2025-07-11 00:00:00+09:00,8730.0,8920.0,8610.0,8650.0,263663,0.0,0.0
2025-07-14 00:00:00+09:00,8670.0,8680.0,8300.0,8370.0,186655,0.0,0.0
2025-07-15 00:00:00+09:00,8350.0,8410.0,8280.0,8400.0,120060,0.0,0.0
2025-07-16 00:00:00+09:00,8320.0,8400.0,8180.0,8200.0,132232,0.0,0.0
2025-07-17 00:00:00+09:00,8200.0,8250.0,8050.0,8250.0,107618,0.0,0.0
2025-07-18 00:00:00+09:00,8260.0,8670.0,8250.0,8560.0,286406,0.0,0.0
2025-07-21 00:00:00+09:00,8620.0,9030.0,8560.0,8940.0,410557,0.0,0.0
2025-07-22 00:00:00+09:00,8900.0,9000.0,8630.0,8670.0,211731,0.0,0.0
2025-07-23 00:00:00+09:00,8790.0,8830.0,8460.0,8710.0,178299,0.0,0.0
